In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass, field, InitVar
import os

# Strategy Evaluation [A017]

## Data

In [2]:
FPATH = "."
FNAME = "Analysis_017.csv"
FFN = os.path.join(FPATH, FNAME)

In [3]:
!ls {FPATH}/*.csv

./Analysis_017.csv


In [4]:
datadf = pd.read_csv(FFN, index_col=0)
datadf

,blockNumber,cid0,tkn0,tkn1,y0_real,z0_real,y1_real,z1_real,p_start0,p_end0,p_start1,p_end1,reason
0,17339692,11570,ETH,DAI,5.0000,5.0000,10000.0000,10000.0000,1750.0,1850.0,1700.0,1500.0,create
1,17339693,11570,ETH,DAI,4.4304,5.0000,11000.0000,11000.0000,1750.0,1850.0,1700.0,1500.0,trade
2,17339694,11570,ETH,DAI,4.9304,5.0000,10153.9659,11000.0000,1750.0,1850.0,1700.0,1500.0,trade
3,17339695,11570,ETH,DAI,4.9304,4.9304,9153.9659,9153.9659,1750.0,1850.0,1700.0,1500.0,user_change
4,17339696,11570,ETH,DAI,3.9304,4.9304,10913.7466,10913.7466,1750.0,1850.0,1700.0,1500.0,trade
5,17339697,11570,ETH,DAI,5.1201,5.1201,8913.7466,10913.7466,1750.0,1850.0,1700.0,1500.0,trade
6,17339698,11570,ETH,DAI,6.1201,6.1201,8913.7466,8913.7466,1750.0,1850.0,1700.0,1500.0,user_change
7,17339699,11570,ETH,DAI,7.1201,7.1201,7233.1899,8913.7466,1750.0,1850.0,1700.0,1500.0,trade
8,17339700,11570,ETH,DAI,6.1201,6.1201,6233.1899,6233.1899,1750.0,1850.0,1700.0,1500.0,user_change
9,17339701,11570,ETH,DAI,5.6201,6.1201,7110.1532,7110.1532,1750.0,1850.0,1700.0,1500.0,trade


## Code

In [5]:
class AttrDict(dict):
    """
    A dictionary that allows for attribute-style access

    see https://stackoverflow.com/questions/4984647/accessing-dict-keys-like-an-attribute
    """

    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

In [6]:
class Prices():
    """
    simple class dealing with token prices
    
    :pricedata:   dict token -> price (in any common numeraire)
    :defaulttkn:  the default quote token for prices
    """
    def __init__(self, pricedata=None, defaulttkn=None, **kwargs):
        if pricedata is None:
            pricedata = dict()
        pricedata = {**pricedata, **kwargs}
        self._pricedata = {k.upper(): v for k,v in pricedata.items()}
        if defaulttkn is None:
            defaulttkn = list(pricedata.keys())[0]
        self.defaulttkn = defaulttkn.upper()
        assert defaulttkn in pricedata, f"defaulttkn [{defaulttkn}] must be in pricedata [{pricedata.keys()}]"
        if not isinstance(pricedata, dict):
            raise ValueError("pricedata must be a dictionary", pricedata)

    def tokens(self):
        """returns set of all tokens"""
        return set(self._pricedata.keys())
    
    def price(self, tknb, tknq=None):
        """
        returns the price of tknb in tknq
        """
        if tknq is None:
            tknq = self.defaulttkn
        return self._pricedata[tknb.upper()] / self._pricedata[tknq.upper()]
    
    def __call__(self, *args, **kwargs):
        """alias for price"""
        return self.price(*args, **kwargs)
    
    def __repr__(self):
        return f"{self.__class__.__name__}(pricedata={self._pricedata}, defaulttkn='{self.defaulttkn}')"
    
P = Prices(usd=1, dai=1, eth=2000)
P

Prices(pricedata={'USD': 1, 'DAI': 1, 'ETH': 2000}, defaulttkn='USD')

In [7]:
@dataclass
class CashFlow():
    """
    represents a single cashflow
    
    :blocknumber:   block number
    :tkn:           token
    :amt:           amount
    """
    blocknumber: int
    tkn: str
    amt: float

In [8]:
@dataclass
class StrategyAnalyzer():
    """
    Analyze performance of Carbon strategies (wrapper object for multiple strategies)
    """
    
    df: InitVar
    datadf: any = field(init=False, repr=False, default=None)
    prices: Prices = field(default=None)

    def __post_init__(self, df):
        df[self.CIDFIELD] = df[self.CIDFIELD].astype(str)
        self.datadf = df
    
    CIDFIELD = "cid0"
    REASONFIELD = "reason"
    RS_CREATE = "create"
    RS_TRADE = "trade"
    RS_CHANGE = "user_change"   
    def cids(self):
        """returns set of all cids"""
        return set(self.datadf[self.CIDFIELD])
    
    BYCID_RAW = "raw"
    BYCID_CHANGES = "changes"
    BYCID_FLOWS = "flows"
    
    def value(self, series, tknq=None):
        """returns the value of the series (in tknq, calculated using self.prices)"""
        val = [amt*self.prices(tkn, tknq) for tkn,amt in zip(series.index, series)]
        return sum(val)
    
    def bycid(self, cid, *, result=None):
        """
        returns dataframe for a given CID only
        
        :cid:      the cid in question
        :result:   BYCID_RAW or BYCID_FLOWS (default)
        :returns:  the requested result
        """
        if result is None:
            result = self.BYCID_FLOWS
            
        df = self.datadf.query(f"{self.CIDFIELD} == '{str(cid)}'").set_index("blockNumber")
        if result == self.BYCID_RAW:
            return df
        
        assert len(df["tkn0"].unique()) == 1, f"must have exactly one tkn0 [{df['tkn0'].unique()}]"
        assert len(df["tkn1"].unique()) == 1, f"must have exactly one tkn1 [{df['tkn1'].unique()}]"
        tkn0 = df["tkn0"].iloc[0]
        tkn1 = df["tkn1"].iloc[0]
        dfd0 = df[["y0_real", "y1_real"]].rename(columns={"y0_real": tkn0, "y1_real": tkn1})
        dfd = dfd0.diff()
        dfd.iloc[0] = dfd0.iloc[0]
        dfd["reason"] = df["reason"]
        assert dfd["reason"].iloc[0] == "create", f"first event must be create [{dfd['reason'].iloc[0]}]"
        events = set(dfd["reason"].iloc[1:])
        assert not "create" in events, f"must not have create event after first [{events}]"
        if result == self.BYCID_CHANGES:
            return dfd
        if result == self.BYCID_FLOWS:
            return dfd.query("reason != 'trade' and reason != 'delete'").drop("reason", axis=1)
        
        raise ValueError("Unknown result", result)

## Analysis

In [9]:
SA = StrategyAnalyzer(datadf, prices=P)

In [10]:
SA.prices

Prices(pricedata={'USD': 1, 'DAI': 1, 'ETH': 2000}, defaulttkn='USD')

In [11]:
analysis_data = AttrDict()
ad = analysis_data

In [12]:
ad.flowdf = SA.bycid(11570)
ad.flowdf

,ETH,DAI
blockNumber,,
17339692,5.0000,10000.0000
17339695,0.0000,-1000.0000
17339698,1.0000,0.0000
17339700,-1.0000,-1000.0000
17339702,0.0000,0.0000
17339705,-2.0000,0.0000
17339706,0.0000,1000.0000
17339707,0.0000,0.0000
17339709,-5.7179,-4833.3734


In [13]:
ad.tknq = P.defaulttkn

In [14]:
ad.initial_amounts = ad.flowdf.iloc[0]
ad.initial_amounts_val = SA.value(ad.initial_amounts)
ad.initial_amounts_val

20000.0

In [15]:
ad.final_amounts = -ad.flowdf.iloc[1:].sum()
ad.final_amounts_val = SA.value(ad.final_amounts)
ad.final_amounts_val

21269.1734

In [16]:
ad.change_amounts = ad.final_amounts - ad.initial_amounts
ad.change_amounts_val = SA.value(ad.change_amounts)
ad.change_amounts_val

1269.1734000000006

In [17]:
ad

{'flowdf':                 ETH         DAI
 blockNumber                    
 17339692     5.0000  10000.0000
 17339695     0.0000  -1000.0000
 17339698     1.0000      0.0000
 17339700    -1.0000  -1000.0000
 17339702     0.0000      0.0000
 17339705    -2.0000      0.0000
 17339706     0.0000   1000.0000
 17339707     0.0000      0.0000
 17339709    -5.7179  -4833.3734,
 'tknq': 'USD',
 'initial_amounts': ETH        5.0
 DAI    10000.0
 Name: 17339692, dtype: float64,
 'initial_amounts_val': 20000.0,
 'final_amounts': ETH       7.7179
 DAI    5833.3734
 dtype: float64,
 'final_amounts_val': 21269.1734,
 'change_amounts': ETH       2.7179
 DAI   -4166.6266
 dtype: float64,
 'change_amounts_val': 1269.1734000000006}